In [32]:
import numpy as np 
import pandas as pd

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
#read the data 
df = pd.read_excel('oedi_data.xlsx', sheet_name='data')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51504 entries, 0 to 51503
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   randid                    51504 non-null  object        
 1   Date                      51504 non-null  datetime64[ns]
 2   NOAAClimRegion            51504 non-null  object        
 3   TempZone                  51504 non-null  object        
 4   HumidZone                 51504 non-null  object        
 5   bin_PlantSize_kW          51504 non-null  object        
 6   plant_age_months          51356 non-null  float64       
 7   active_snow_tickets       51504 non-null  object        
 8   snow_bin_ticket_minutes   790 non-null    object        
 9   snow_affected_assets      790 non-null    object        
 10  snow_production_level     51437 non-null  object        
 11  PR                        49940 non-null  float64       
 12  snow_value_mm     

In [36]:
#filter our hawaii and south regions 
df = df[(df.NOAAClimRegion != 'Hawaii') & (df.NOAAClimRegion != 'South')]

#filter for date
start = '2018-08-01'
end = '2019-04-30'

clean_df = df[(df['Date'] >= start) & (df['Date'] <= end)]
clean_df

,randid,Date,NOAAClimRegion,TempZone,HumidZone,bin_PlantSize_kW,plant_age_months,active_snow_tickets,snow_bin_ticket_minutes,snow_affected_assets,snow_production_level,PR,snow_value_mm,total_daily_snow_mm,low_irradiation,cumulative_snow_mm,hurr_bin_ticket_minutes,hurr_affected_assets,hurr_production_level,HurricanePrep,HurricanePostInspection,hurricane,nearest_hurricane,wind_speed_mean,rain_value_mm,nearest_rain,storm_active_tickets,storm_bin_ticket_minutes,storm_affected_assets,storm_production_level,lightning,storm,duration_minutes_storm,nearest_storm,flood,duration_minutes_flood,nearest_flood,rain
122,C2S1,2018-08-01,West,T6,H4,large,23.0,No,NaN,NaN,Unknown,0.528309,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,6.795042,0.000000,13,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,0.0
123,C2S1,2018-08-02,West,T6,H4,large,23.0,No,NaN,NaN,Unknown,0.720385,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,6.854104,0.000000,14,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,0.0
124,C2S1,2018-08-03,West,T6,H4,large,23.0,No,NaN,NaN,Unknown,0.732191,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,6.453604,0.000000,15,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,0.0
125,C2S1,2018-08-04,West,T6,H4,large,23.0,No,NaN,NaN,Unknown,0.733979,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,5.954604,0.000000,16,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,0.0
126,C2S1,2018-08-05,West,T6,H4,large,23.0,No,NaN,NaN,Unknown,0.728649,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,6.222104,0.000000,17,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51407,C3S40,2019-04-26,Southeast,T5,H4,medium,28.0,No,NaN,NaN,Unknown,0.714535,0.0,0.0,0,0.0,Majority,Other,Partial,No,No,0.0,192,3.217698,0.000000,6,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,223,0.0
51408,C3S40,2019-04-27,Southeast,T5,H4,medium,28.0,No,NaN,NaN,Unknown,0.507682,0.0,0.0,0,0.0,Majority,Other,Partial,No,No,0.0,193,1.549315,7.624962,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,224,1.0
51409,C3S40,2019-04-28,Southeast,T5,H4,medium,28.0,No,NaN,NaN,Unknown,0.756540,0.0,0.0,0,0.0,Majority,Other,Partial,No,No,0.0,194,1.844885,0.000000,1,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,225,0.0
51410,C3S40,2019-04-29,Southeast,T5,H4,medium,28.0,No,NaN,NaN,Unknown,0.776597,0.0,0.0,0,0.0,Majority,Other,Partial,No,No,0.0,195,2.237012,0.000000,2,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,226,0.0


In [37]:
#review all nulls 
clean_df.isnull().sum()

randid                          0
Date                            0
NOAAClimRegion                  0
TempZone                        0
HumidZone                       0
bin_PlantSize_kW                0
plant_age_months              148
active_snow_tickets             0
snow_bin_ticket_minutes     34963
snow_affected_assets        34963
snow_production_level          67
PR                            996
snow_value_mm                3098
total_daily_snow_mm          3423
low_irradiation                 0
cumulative_snow_mm           5947
hurr_bin_ticket_minutes     35122
hurr_affected_assets        35122
hurr_production_level          17
HurricanePrep                   0
HurricanePostInspection         0
hurricane                    3080
nearest_hurricane               0
wind_speed_mean              9842
rain_value_mm                6605
nearest_rain                    0
storm_active_tickets            0
storm_bin_ticket_minutes    35261
storm_affected_assets       35261
storm_producti

In [38]:
custom_fill = {
    'snow_bin_ticket_minutes': 'None',
    'snow_affected_assets': 'None',
    'snow_production_level': 'Unknown',
    'hurr_bin_ticket_minutes': 'None',
    'hurr_affected_assets': 'None',
    'hurr_production_level': 'Unknown',
    'storm_bin_ticket_minutes': 'None',
    'storm_affected_assets': 'None',
    'storm_production_level': 'Unknown'
}

clean_df.fillna(value=custom_fill, inplace=True)

for col in clean_df.columns:
    if col not in custom_fill:
        if clean_df[col].dtype == 'object':
            clean_df[col] = clean_df[col].fillna('No')
        elif np.issubdtype(clean_df[col].dtype, np.integer):
            clean_df[col] = clean_df[col].fillna(0)
        elif np.issubdtype(clean_df[col].dtype, np.floating):
            clean_df[col] = clean_df[col].fillna(0.0)

/var/folders/8m/pk0njzd50fn7mbqbtt0zrdph0000gn/T/ipykernel_42646/1883341550.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.fillna(value=custom_fill, inplace=True)
/var/folders/8m/pk0njzd50fn7mbqbtt0zrdph0000gn/T/ipykernel_42646/1883341550.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[col] = clean_df[col].fillna('No')
/var/folders/8m/pk0njzd50fn7mbqbtt0zrdph0000gn/T/ipykernel_42646/1883341550.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [39]:
clean_df.isnull().sum()

randid                      0
Date                        0
NOAAClimRegion              0
TempZone                    0
HumidZone                   0
bin_PlantSize_kW            0
plant_age_months            0
active_snow_tickets         0
snow_bin_ticket_minutes     0
snow_affected_assets        0
snow_production_level       0
PR                          0
snow_value_mm               0
total_daily_snow_mm         0
low_irradiation             0
cumulative_snow_mm          0
hurr_bin_ticket_minutes     0
hurr_affected_assets        0
hurr_production_level       0
HurricanePrep               0
HurricanePostInspection     0
hurricane                   0
nearest_hurricane           0
wind_speed_mean             0
rain_value_mm               0
nearest_rain                0
storm_active_tickets        0
storm_bin_ticket_minutes    0
storm_affected_assets       0
storm_production_level      0
lightning                   0
storm                       0
duration_minutes_storm      0
nearest_st

In [40]:
clean_df['weather_onset'] = (
    (clean_df['active_snow_tickets'] == 'Yes') |
    (clean_df['snow_bin_ticket_minutes'] == 'Yes') |
    (clean_df['hurricane'] == 1) |
    (clean_df['storm'] == 1) |
    (clean_df['flood'] == 1) |
    (clean_df['rain'] == 1) |
    (clean_df['lightning'] == 1)
).astype(int)

clean_df.head(5)

/var/folders/8m/pk0njzd50fn7mbqbtt0zrdph0000gn/T/ipykernel_42646/2686608608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['weather_onset'] = (


,randid,Date,NOAAClimRegion,TempZone,HumidZone,bin_PlantSize_kW,plant_age_months,active_snow_tickets,snow_bin_ticket_minutes,snow_affected_assets,snow_production_level,PR,snow_value_mm,total_daily_snow_mm,low_irradiation,cumulative_snow_mm,hurr_bin_ticket_minutes,hurr_affected_assets,hurr_production_level,HurricanePrep,HurricanePostInspection,hurricane,nearest_hurricane,wind_speed_mean,rain_value_mm,nearest_rain,storm_active_tickets,storm_bin_ticket_minutes,storm_affected_assets,storm_production_level,lightning,storm,duration_minutes_storm,nearest_storm,flood,duration_minutes_flood,nearest_flood,rain,weather_onset
122,C2S1,2018-08-01,West,T6,H4,large,23.0,No,None,None,Unknown,0.528309,0.0,0.0,0,0.0,None,None,Unknown,No,No,0.0,354,6.795042,0.0,13,No,None,None,Unknown,0,0.0,0,2420,0.0,0,2255,0.0,0
123,C2S1,2018-08-02,West,T6,H4,large,23.0,No,None,None,Unknown,0.720385,0.0,0.0,0,0.0,None,None,Unknown,No,No,0.0,354,6.854104,0.0,14,No,None,None,Unknown,0,0.0,0,2420,0.0,0,2255,0.0,0
124,C2S1,2018-08-03,West,T6,H4,large,23.0,No,None,None,Unknown,0.732191,0.0,0.0,0,0.0,None,None,Unknown,No,No,0.0,354,6.453604,0.0,15,No,None,None,Unknown,0,0.0,0,2420,0.0,0,2255,0.0,0
125,C2S1,2018-08-04,West,T6,H4,large,23.0,No,None,None,Unknown,0.733979,0.0,0.0,0,0.0,None,None,Unknown,No,No,0.0,354,5.954604,0.0,16,No,None,None,Unknown,0,0.0,0,2420,0.0,0,2255,0.0,0
126,C2S1,2018-08-05,West,T6,H4,large,23.0,No,None,None,Unknown,0.728649,0.0,0.0,0,0.0,None,None,Unknown,No,No,0.0,354,6.222104,0.0,17,No,None,None,Unknown,0,0.0,0,2420,0.0,0,2255,0.0,0


In [41]:
#export to csv
clean_df.to_csv('PV_data_clean.csv', index=False)